In [52]:
import pandas as pd 
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, Flatten, Dropout, Concatenate
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.layers import LSTM, Bidirectional
from keras.models import Model, Sequential, load_model
from keras.callbacks import EarlyStopping
import gensim
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import re
import codecs
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
from gensim.models.ldamodel import LdaModel
from gensim.corpora import Dictionary
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')


In [100]:
import gensim
from nltk.corpus import wordnet

from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
import re
stopWords = set(STOPWORDS)
lmtzr = nltk.WordNetLemmatizer().lemmatize


def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def normalize_text(text):
    token_text = nltk.word_tokenize(text)
    cleaned_token_text = []
    for tt in token_text:
        if tt in stopWords or tt == '' or len(tt) < 2: continue    
        cleaned_token_text.append(tt)
    
    word_pos = nltk.pos_tag(cleaned_token_text)
    lemm_words = [lmtzr(sw[0], get_wordnet_pos(sw[1])) for sw in word_pos]
    
    return [x.lower() for x in lemm_words]

# def clean_text(text):
#     text = re.sub("[^a-zA-Z']", ' ', text)
#     cleaned_txt = ''
#     for word in normalize_text(text):
#         cleaned_txt += (word + ' ')
#     return cleaned_txt

In [101]:
d = {'text': ['when thieves broke into my house at night, and held my wife and me on gun point for at least ten minutes and took away a lot of property.',
                 'International sports events won by my favourite national team or player brings me joy when india won the world cup cricket match.',
                 'I was sitting in a restaurant with friends. They asked me something which they thought I should know. Actually I know it but at that time I was not able to remember it.']}
d_ex = pd.DataFrame(data=d)

In [102]:
with pd.option_context('display.max_colwidth', 500):
    print(d_ex)

                                                                                                                                                                       text
0                                 when thieves broke into my house at night, and held my wife and me on gun point for at least ten minutes and took away a lot of property.
1                                         International sports events won by my favourite national team or player brings me joy when india won the world cup cricket match.
2  I was sitting in a restaurant with friends. They asked me something which they thought I should know. Actually I know it but at that time I was not able to remember it.


In [103]:
# d_ex['cleaned_text'] = d_ex['text'].apply(lambda x: clean_text(x))
# with pd.option_context('display.max_colwidth', 500):
#     print(d_ex['cleaned_text'])

In [104]:
lda_model = LdaModel.load('lda_model')

In [105]:
dictionary = Dictionary.load('lda_dict')

In [138]:
sug_words_list = []
for text in d_ex['text']:
    doc_bow = dictionary.doc2bow(normalize_text(text))
    doc_lda = lda_model[doc_bow]
    temp = []
    for index, score in sorted(doc_lda, key=lambda tup: -1*tup[1]):
        for word, score in lda_model.show_topic(index, 4):
            temp.append(word)
    sug_words_list.append(temp)

sug_words = pd.Series(sug_words_list)
d_ex['suggested_words'] = sug_words

In [139]:
cnn_model = load_model('cnn_model.h5')

In [140]:
import pickle
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [141]:
MAX_SEQUENCE_LENGTH = 150
y_predict = cnn_model.predict(pad_sequences(tokenizer.texts_to_sequences(d_ex['text'].tolist()), maxlen=MAX_SEQUENCE_LENGTH)
, batch_size=64, verbose=1)

3/3 [==============================] - 0s 108ms/step


In [142]:
y_predict_top1 = []
for i in range(0, len(y_predict)):
    max_index = 0
    max_value = 0.0
    for j in range(0, len(y_predict[0])):
        if y_predict[i][j] > max_value:
            max_value = y_predict[i][j]
            max_index = j

    y_predict_top1.append(max_index)

In [143]:
suggested_emoji = pd.Series(y_predict_top1)

In [144]:
d_ex['suggested_emoji'] = suggested_emoji

In [145]:
with pd.option_context('display.max_colwidth', 500):
    print(d_ex)

                                                                                                                                                                       text  \
0                                 when thieves broke into my house at night, and held my wife and me on gun point for at least ten minutes and took away a lot of property.   
1                                         International sports events won by my favourite national team or player brings me joy when india won the world cup cricket match.   
2  I was sitting in a restaurant with friends. They asked me something which they thought I should know. Actually I know it but at that time I was not able to remember it.   

                                                                                                                                                                                                                                                                suggested_words  \
0                       